In [1]:
import os 
os.chdir('F:\VIP CUP\VIP_CUP_Task2')
import cv2
import numpy as np
import os
import sys
import coco
import utils
import model as modellib
import time


#if we use ImageNet model for any specific classifier
'''
from keras.applications import InceptionV3
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
'''

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [2]:
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


config = InferenceConfig()

#config.display()

model = modellib.MaskRCNN(
    mode="inference", model_dir=MODEL_DIR, config=config
)
model.load_weights(COCO_MODEL_PATH, by_name=True)
class_names = [
    'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
    'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    'teddy bear', 'hair drier', 'toothbrush'
]


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
def video_from_dir(dir):
    temp=os.listdir(dir)   #give input directory
    video=[]
    for i in temp:
        if(i.endswith('.MP4')):
            video.append(i)
    return video

In [4]:
# Main defining function for blurry image
def blur_image(img):
    """
    Args:
        img (numpy.array): RGB image. Contains people. Used as "read-only" and will not be modified by this function.
        img_background (numpy.array): RGB image. There are no people in this image. 
    """
    start = time.time()
    
    # img_copy will be modified with the pixel manipulations and returned by the function
    img_copy = np.copy(img)
    
    blur_img=cv2.GaussianBlur(img,(45,45),0)  ##BLUR## 
    
    results = model.detect([img], verbose=0)
    results = results[0]
    
    '''
    ##NOT NEEDED as it shows image with every classifiers
    if display_imgs:
        visualize.display_instances(img, results['rois'], results['masks'], results['class_ids'], 
                                    class_names, results['scores'])
    '''
    # shape of the masks are (h, w, object_ix)
    n_objects_found = len(results['class_ids'])
    for ix in range(n_objects_found):
        if ((results['class_ids'][ix] == (class_names.index('tv'))) 
            or (results['class_ids'][ix] == (class_names.index('person'))) or (results['class_ids'][ix] == (class_names.index('laptop'))) 
            or (results['class_ids'][ix] == (class_names.index('keyboard'))) 
            or (results['class_ids'][ix] == (class_names.index('cell phone')))
           or (results['class_ids'][ix] == (class_names.index('toilet')))) :      ###for some specific sensitive classes
            bbox = results['rois'][ix]
            if bbox is not None:
                x, y, w, h = bbox          ###get a different image for person face depending on bbox
                
                p_mask = results['masks'][:, :, ix]       ###get the specific object's mask

                # The mask is reduced by 1 pixels in all four directions toensure the object is precisely covered.
                ##(NEEDED FOR PRECISION)
                padding = 1
                p_mask_2 = np.roll(p_mask, -padding, axis=0)
                p_mask_3 = np.roll(p_mask, -padding, axis=0)
                p_mask_4 = np.roll(p_mask, -padding, axis=1)
                p_mask_5 = np.roll(p_mask, -padding, axis=1)
                p_masks = [p_mask, p_mask_2, p_mask_3, p_mask_4, p_mask_5]
                p_masks_merged = np.logical_or.reduce(p_masks, axis=0)

                other_masks = np.delete(results['masks'], ix, axis=2)
                other_masks_merged = np.logical_or.reduce(other_masks, axis=2)

                person_mask_minus_others = p_masks_merged & np.logical_not(other_masks_merged)

                '''MAIN PROCESSING PART
                to blur each frame'''

                #img_copy[person_mask_minus_others] = img_background[person_mask_minus_others]   #to make invisible
                img_copy[person_mask_minus_others] = blur_img[person_mask_minus_others]

    end = time.time()
    print('{} seconds to process image'.format(int(end - start)))
    
    return img_copy


In [ ]:
if __name__ == '__main__':
    cur_dir=os.getcwd()
    input_path=os.path.join(cur_dir,'nextbatchRun')         #whatever test_directory is named
    video=video_from_dir(input_path)
    for j in range(len(video)):
        print(video[j])  
        capture = cv2.VideoCapture(os.path.join(input_path,video[j]))
        fps = capture.get(cv2.CAP_PROP_FPS)
        size = (
        int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        )
        codec = cv2.VideoWriter_fourcc(*'MP4V')
        os.chdir(os.path.join(cur_dir,'output'))          #whatever protected saved data folder name is 
        output = cv2.VideoWriter(video[j], codec, fps, size)
        ret, frame = capture.read()
        while(ret):
            results = model.detect([frame], verbose=0)
            r = results[0]
            img_processed = blur_image(frame)
            output.write(img_processed)
            #cv2.imshow('frame',img_processed)
            if cv2.waitKey(0) & 0xFF == ord('q'):
                break
            ret, frame = capture.read()

        capture.release()
        output.release()
        cv2.destroyAllWindows()


12_21_chat.MP4
14 seconds to process image
13 seconds to process image
14 seconds to process image
13 seconds to process image
14 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
14 seconds to process image
13 seconds to process image
14 seconds to process image
13 seconds to process image
14 seconds to process image
13 seconds to process image
13 seconds to process image
14 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
14 seconds to process image
14 seconds to process image
14 seconds to process image
13 seconds to process image
14 seconds to process image
13 seconds to process image
13 seconds to process image
13 seconds to process image
1_24_shake.MP4
13 seconds to process image
13 seconds to process image
13 seconds to proc